In [ ]:
!pip install --user -q transformers accelerate bitsandbytes \
datasets evaluate sentence-transformers faiss-gpu langchain \
langchain-community openpyxl pacmap ragatouille langchain-huggingface rank_bm25 gdown

!pip install -U ipywidgets

In [ ]:
!nvidia-smi

In [ ]:
import os
os.environ["PATH"] += ":/root/.local/bin"

In [ ]:
!pip install -U langchain-community
# !pip install vllm

In [ ]:
# 降级transformer
# !pip install transformers==4.30.2
!pip install vllm

In [2]:
# Import standard Python tools
import os
import io
import pickle
import contextlib
from typing import Optional, List, Tuple
import json
import torch
# from rank_bm25 import BM25Okapi
from concurrent.futures import ThreadPoolExecutor

# Import libraries for data handling and visualization
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.io as pio

# Configure pandas for better visualization of retriever outputs
pd.set_option("display.max_colwidth", None)

# Import text processing and embeddings tools
from sentence_transformers import SentenceTransformer
from transformers import (
    pipeline,
    Pipeline,
    AutoTokenizer,
    AutoModelForCausalLM,
    AutoModelForQuestionAnswering,
    BitsAndBytesConfig,
    DefaultDataCollator,
    TrainingArguments,
    Trainer,
)

# Import LangChain tools
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain.docstore.document import Document as LangchainDocument
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores.utils import DistanceStrategy
# from langchain_community.vectorstores.utils import DistanceStrategy

# Import RAG-specific tools
# from ragatouille import RAGPretrainedModel

# Import tools for datasets
from datasets import Dataset

# Advanced visualization and dimensionality reduction tools
import pacmap

# Progress bar for loops
from tqdm.notebook import tqdm

# Import vLLM
from vllm import LLM, SamplingParams

In [21]:
if not os.path.exists("/kaggle/working/triviaqa-rc"):
    # Download the dataset in the working space of Kaggle
    !wget https://nlp.cs.washington.edu/triviaqa/data/triviaqa-rc.tar.gz
    # Create a directory for extracting the content
    !mkdir /kaggle/working/triviaqa-rc
    # Extract the content from the downloaded file
    !tar -xzf /kaggle/working/triviaqa-rc.tar.gz -C /kaggle/working/triviaqa-rc
    # Delete the compressed file
    !rm /kaggle/working/triviaqa-rc.tar.gz

--2025-04-15 20:03:23--  https://nlp.cs.washington.edu/triviaqa/data/triviaqa-rc.tar.gz
Resolving nlp.cs.washington.edu (nlp.cs.washington.edu)... 128.208.3.117, 2607:4000:200:12:3eec:efff:fe5e:6f68
Connecting to nlp.cs.washington.edu (nlp.cs.washington.edu)|128.208.3.117|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2665779500 (2.5G) [application/x-gzip]
Saving to: ‘triviaqa-rc.tar.gz’

triviaqa-rc.tar.gz  100%[===================>]   2.48G   105MB/s    in 27s     

2025-04-15 20:03:50 (93.1 MB/s) - ‘triviaqa-rc.tar.gz’ saved [2665779500/2665779500]



In [22]:
# Load the original training file
train_data = pd.read_json("/kaggle/working/triviaqa-rc/qa/wikipedia-train.json")

# Split the data: first 7900 questions for validation, the rest for training
validation_data = train_data.iloc[:7900]
train_data = train_data.iloc[7900:] # We shouldn't use this (it's not necessary if we are using a RAG system)

# Load the original validation file to use as test data
test_data = pd.read_json("/kaggle/working/triviaqa-rc/qa/wikipedia-dev.json") # The actual test data is hidden

print("Data partitioned successfully:")
print(f"Training set size: {len(train_data)}")
print(f"Validation set size: {len(validation_data)}")
print(f"Test set size: {len(test_data)}")

Data partitioned successfully:
Training set size: 53988
Validation set size: 7900
Test set size: 7993


In [ ]:
validation_data.head(3)

In [ ]:
#Build a list of docs from the downloaded Wikipedia documents
def getDocs(examples):
    # List of docs
    docs = []
    # Extracting metadata and filenames from the examples (training data)
    for example in examples["Data"]:
        filename = example["EntityPages"][0]["Filename"]
        with open(f"triviaqa-rc/evidence/wikipedia/{filename}", "r") as file:
            context_text = file.read()
        # Create a Document for RAG
        newDoc = LangchainDocument(
                metadata={
                    'question_id': example['QuestionId'],
                    'source': example['EntityPages'][0]['DocSource'],
                    'answer_type': example['Answer']['Type'],
                    'entity_name': example['Answer'].get('NormalizedValue', ''), # Default to empty string if missing
                    'aliases': example['Answer'].get('Aliases', []), # Default to empty list if missing
                    'normalized_value': example['Answer'].get('NormalizedValue', ''), # Default to empty string if missing
                    'filename': example['EntityPages'][0]['Filename'],
                },
                page_content=context_text
        )
        docs.append(newDoc)
    return docs

In [ ]:
# Use the validation data to decide the size of each document chunk
RAW_KNOWLEDGE_BASE = getDocs(validation_data)

# Use the test data to create the knowledge base
#RAW_KNOWLEDGE_BASE = getDocs(test_data)

In [ ]:
# We use a hierarchical list of separators specifically tailored for splitting Markdown documents
# This list is taken from LangChain's MarkdownTextSplitter class
# 在这里选择chunk大小
'''
MARKDOWN_SEPARATORS = [
    "\n#{1,6} ",
    "```\n",
    "\n\\*\\*\\*+\n",
    "\n---+\n",
    "\n___+\n",
    "\n\n",
    "\n",
    " ",
    "",
]
'''
MARKDOWN_SEPARATORS = [
    "\n\n",  # Paragraph breaks
    "\n",    # Line breaks
    ". ",    # Sentences
    "? ",    # Questions
    "! ",    # Exclamations
    "; ",    # Semicolons
    ": ",    # Colons
    ", ",    # Commas
    " ",     # Words
    ""       # Characters
]
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, # The maximum number of characters in a chunk: we selected this value arbitrarily 
    chunk_overlap=100, # The number of characters to overlap between chunks
    add_start_index=True, # If `True`, includes chunk's start index in metadata
    strip_whitespace=True, # If `True`, strips whitespace from the start and end of every document
    separators=MARKDOWN_SEPARATORS,
)

docs_processed = []
for doc in RAW_KNOWLEDGE_BASE: # We are using the validation data to test
    docs_processed += text_splitter.split_documents([doc])

In [ ]:
# To get the value of the max sequence_length, we will query the underlying `SentenceTransformer` object used in the RecursiveCharacterTextSplitter
print(
    f"Model's maximum sequence length: {SentenceTransformer('thenlper/gte-small').max_seq_length}"
)

tokenizer = AutoTokenizer.from_pretrained("thenlper/gte-small")
lengths = [len(tokenizer.encode(doc.page_content)) for doc in tqdm(docs_processed)]

# Plot the distribution of document lengths, counted as the number of tokens
fig = pd.Series(lengths).hist()
plt.title("Distribution of document lengths in the knowledge base (in count of tokens)")
plt.show()

In [ ]:
EMBEDDING_MODEL_NAME = "thenlper/gte-small" # The name of the SentenceTransformer
# gte-small和一个LLM最前面的一步差不多字，token，embedding取出tokenization
# Function to split the documents into chunks
def split_documents(
    chunk_size: int,
    knowledge_base: List[LangchainDocument],
    tokenizer_name: Optional[str] = EMBEDDING_MODEL_NAME,
) -> List[LangchainDocument]:
    """
    Split documents into chunks of maximum size `chunk_size` tokens and return a list of documents.
    """

    '''
    MARKDOWN_SEPARATORS = [
        "\n#{1,6} ",
        "```\n",
        "\n\\*\\*\\*+\n",
        "\n---+\n",
        "\n___+\n",
        "\n\n",
        "\n",
        " ",
        "",
    ]
    '''
    MARKDOWN_SEPARATORS = [
        "\n\n",  # Paragraph breaks
        "\n",    # Line breaks
        ". ",    # Sentences
        "? ",    # Questions
        "! ",    # Exclamations
        "; ",    # Semicolons
        ": ",    # Colons
        ", ",    # Commas
        " ",     # Words
        ""       # Characters
    ]

    text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
        AutoTokenizer.from_pretrained(tokenizer_name),
        chunk_size=chunk_size,
        chunk_overlap=int(chunk_size / 10),
        add_start_index=True,
        strip_whitespace=True,
        separators=MARKDOWN_SEPARATORS,
    )

    docs_processed = []
    for doc in knowledge_base:
        docs_processed += text_splitter.split_documents([doc])

    # Remove duplicates
    unique_texts = {}
    docs_processed_unique = []
    for doc in docs_processed:
        if doc.page_content not in unique_texts:
            unique_texts[doc.page_content] = True
            docs_processed_unique.append(doc)

    return docs_processed_unique

# Processing the RAW_KNOWLEDGE_BASE (validation_data)
docs_processed = split_documents(
    512, # We choose a chunk size adapted to our model
    RAW_KNOWLEDGE_BASE,
    tokenizer_name=EMBEDDING_MODEL_NAME,
)

# Save the object
with open("/kaggle/working/docs_processed.pkl", "wb") as file:
    pickle.dump(docs_processed, file)

tokenizer = AutoTokenizer.from_pretrained(EMBEDDING_MODEL_NAME)
lengths = [len(tokenizer.encode(doc.page_content)) for doc in tqdm(docs_processed)]
fig = pd.Series(lengths).hist()
plt.title("Distribution of document lengths in the knowledge base (in count of tokens)")
plt.show()

In [3]:
# The name of the embedding model
EMBEDDING_MODEL_NAME = "thenlper/gte-small"
# HuggingFace Embedding Model
embedding_model = HuggingFaceEmbeddings(
    model_name=EMBEDDING_MODEL_NAME,
    multi_process=True,
    model_kwargs={"device": "cuda"},
    encode_kwargs={"normalize_embeddings": True},  # Set `True` for cosine similarity
)

modules.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/68.1k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/66.7M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/394 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# This cell below takes around half an hour to run
# 不用跑 
# # Create the vector database of document embeddings
KNOWLEDGE_VECTOR_DATABASE = FAISS.from_documents(
    docs_processed, embedding_model, distance_strategy=DistanceStrategy.COSINE
)

# #Save the vector database in a file
faiss_index_path = "/kaggle/working/knowledge_vector_database-validation"


In [ ]:
# 加载上传的本地数据集要跑
# Load the pre-created vector database
faiss_index_path = "/kaggle/input/dataset/knowledge_vector_database-validation"
# faiss_index_path = "/kaggle/working/knowledge-vector-database-validation"
KNOWLEDGE_VECTOR_DATABASE = FAISS.load_local(faiss_index_path, embedding_model, allow_dangerous_deserialization=True)

In [ ]:
# KNOWLEDGE_VECTOR_DATABASE.save_local(faiss_index_path)
# 这个好像也不用跑

In [ ]:
# # Embed a user query in the same space
user_query = "Which Mediterranean island was once known as Alashiya?"
query_vector = embedding_model.embed_query(user_query)

In [ ]:
embedding_projector = pacmap.PaCMAP(
    n_components=2, n_neighbors=None, MN_ratio=0.5, FP_ratio=2.0, random_state=1
)

embeddings_2d = [
    list(KNOWLEDGE_VECTOR_DATABASE.index.reconstruct_n(idx, 1)[0])
    for idx in range(len(docs_processed))
] + [query_vector]

# Fit the data (the index of transformed data corresponds to the index of the original data)
documents_projected = embedding_projector.fit_transform(
    np.array(embeddings_2d), init="pca"
)

In [ ]:
pio.renderers.default = 'iframe'

In [ ]:
# Create DataFrame for document embeddings 
df = pd.DataFrame.from_dict(
    [
        {
            "x": documents_projected[i, 0],  # Accessing document projection
            "y": documents_projected[i, 1],
            "source": docs_processed[i].metadata["source"],
            "extract": docs_processed[i].page_content[:100] + "...",
            "symbol": "circle",
            "size_col": 4,
        }
        for i in range(len(docs_processed))  # Only iterate over document embeddings
    ]
    + [
        {
            "x": documents_projected[-1, 0],  # Append the query vector
            "y": documents_projected[-1, 1],
            "source": "User query",
            "extract": user_query,
            "size_col": 100,
            "symbol": "star",
        }
    ]
)

# Visualize the embedding
fig = px.scatter(
    df,
    x="x",
    y="y",
    color="source",
    text="extract",
    symbol="symbol",
    size="size_col",
    title="2D Projection of Documents and Query",
    width=1000,
    height=700,
)
fig.show()

In [ ]:
print(f"\nStarting retrieval for {user_query=}...")
retrieved_docs = KNOWLEDGE_VECTOR_DATABASE.similarity_search(query=user_query, k=5)
print(
    "\n==================================Top 1 document=================================="
)
print(retrieved_docs[0].page_content)
print("==================================Metadata==================================")
print(retrieved_docs[0].metadata)
print(
    "\n==================================Top 2 document=================================="
)
print(retrieved_docs[1].page_content)
print("==================================Metadata==================================")
print(retrieved_docs[1].metadata)

In [4]:
from rank_bm25 import BM25Okapi

In [ ]:
# Extract the page content from the documents
doc_texts = [doc.page_content for doc in docs_processed]

# Tokenize the document texts
tokenized_corpus = [text.split() for text in doc_texts]

# Initialize BM25 retriever
bm25 = BM25Okapi(tokenized_corpus)

# File path for saving the BM25 retriever
bm25_file_path = '/kaggle/working/bm25_retriever_validation.pkl'

# Save the tokenized corpus
with open(bm25_file_path, 'wb') as f:
    pickle.dump(tokenized_corpus, f)

In [ ]:
# Instead
# 这个不跑
# Load the object
with open("/kaggle/input/docs-processed-validation/docs_processed.pkl", "rb") as file:
    docs_processed = pickle.load(file)

# Load the tokenized corpus
with open('/kaggle/input/bm25-retriever-validation/bm25_retriever_validation.pkl', 'rb') as f:
    loaded_tokenized_corpus = pickle.load(f)

# Reinitialize BM25 retriever using the loaded corpus
bm25 = BM25Okapi(loaded_tokenized_corpus)

In [5]:
def bm25_retrieve(query, k=5):
    """
    Perform retrieval using BM25 on LangChain documents.

    Args:
        query (str): The search query.
        k (int): The number of top documents to retrieve.

    Returns:
        list: A list of tuples with document indices, scores, and the documents themselves.
    """
    # Tokenize the query
    tokenized_query = query.split()

    # Get scores for all documents
    scores = bm25.get_scores(tokenized_query)

    # Get top-k document indices and scores
    top_k_indices = np.argsort(scores)[::-1][:k]
    top_k_results = [(index, scores[index], docs_processed[index]) for index in top_k_indices]

    return top_k_results

In [ ]:
# Example query
query = "Which Lloyd Webber musical premiered in the US on 10th December 1993?"
top_docs = bm25_retrieve(query, k=3)

# Display results
for idx, score, doc in top_docs:
    print("================================================================================================")
    print(f"Doc Index: {idx}, Score: {score}, Content: {doc.page_content}, Metadata: {doc.metadata}")

In [6]:
!pip install together

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.9/87.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 63.4 MB/s eta 0:00:00:00:010:01
  Attempting uninstall: pillow
    Found existing installation: pillow 11.0.0
    Uninstalling pillow-11.0.0:
      Successfully uninstalled pillow-11.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastai 2.7.18 requires torch<2.6,>=1.10, but you have torch 2.6.0 which is incompatible.
mlxtend 0.23.3 requires scikit-learn>=1.3.1, but you have scikit-learn 1.2.2 which is incompatible.
plotnine 0.14.4 requires matplotlib>=3.8.0, but you have matplotlib 3.7.5 which is incompatible.


In [7]:
from together import Together
import tiktoken
import numpy as np
from typing import List

# 初始化 Together API 客户端
client = Together(api_key="API_KEY")

# 获取 Llama-3.1-8B-Instruct-Turbo 的 tokenizer
enc = tiktoken.get_encoding("cl100k_base")

# Sampling 参数类
class SamplingParams:
    def __init__(self, n=1, top_p=0.9, temperature=0.7, repetition_penalty=1.2, max_tokens=256):
        self.n = n
        self.top_p = top_p
        self.temperature = temperature
        self.repetition_penalty = repetition_penalty
        self.max_tokens = max_tokens

# 自定义 Tokenizer 代理类
class TogetherTokenizer:
    """ 模拟 transformers.Tokenizer，提供 `apply_chat_template` 和 `tokenize` 以兼容原代码 """

    def __init__(self):
        pass

    def apply_chat_template(self, messages, tokenize=False, add_generation_prompt=True):
        """ 模拟 `apply_chat_template`，将聊天格式转换为文本 """
        chat_template = ""
        for msg in messages:
            role = msg.get("role", "user")
            content = msg.get("content", "")
            chat_template += f"{role.capitalize()}: {content}\n"

        if add_generation_prompt:
            chat_template += "Assistant: "  # 让 LLM 继续生成回答
        
        return chat_template if not tokenize else chat_template.split()

    def tokenize(self, text):
        """ 使用 `tiktoken` 进行 tokenization """
        return enc.encode(text)

# Together LLM Reader
class TogetherLLMReader:
    def __init__(self, model_name="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo", api_key="API_KEY"):
        self.client = Together(api_key=api_key)
        self.model_name = model_name
        self.tokenizer = TogetherTokenizer()

    def get_tokenizer(self):
        """ 兼容原代码，返回自定义 tokenizer """
        return self.tokenizer

    def generate(self, prompts: List[str], sampling_params: SamplingParams):
        """ 兼容原代码，支持批量 Prompt 生成 """
        responses = []
        for prompt in prompts:
            response = self.client.chat.completions.create(
                model=self.model_name,
                messages=[{"role": "user", "content": prompt}],
                temperature=sampling_params.temperature,
                max_tokens=sampling_params.max_tokens,
                top_p=sampling_params.top_p,
                repetition_penalty=sampling_params.repetition_penalty,
                n=sampling_params.n,
            )
            responses.append(response.choices[0].message.content)

        # 兼容原代码：包装返回结果
        return [LLMOutput(prompt, generated_text) for prompt, generated_text in zip(prompts, responses)]

# 兼容原代码的输出格式
class LLMOutput:
    def __init__(self, prompt, text):
        self.prompt = prompt
        self.outputs = [LLMResponse(text)]

class LLMResponse:
    def __init__(self, text):
        self.text = text


In [ ]:

# Define the model
# from together import Together
'''
READER_MODEL_NAME = "/kaggle/input/zephyr-7b-beta-awq"
#READER_MODEL_NAME = "/kaggle/input/hugging-quants-meta-llama-3-1-8b-instruct-awq-int4"
#READER_MODEL_NAME = "PyrTools/Ministral-8B-Instruct-2410-AWQ"

# Configure the model

model = LLM( 
    model = READER_MODEL_NAME,
    quantization="awq",
    tensor_parallel_size=2, 
    gpu_memory_utilization=0.95, 
    trust_remote_code=True,
    dtype="half", 
    enforce_eager=True,
    #max_model_len=1024,
    disable_log_stats=True
)

tokenizer = model.get_tokenizer()
'''

In [8]:
# 初始化模型
model = TogetherLLMReader(api_key="API_KEY")

# 获取 tokenizer
tokenizer = model.get_tokenizer()

In [ ]:
# Initialize a variable to accumulate the average token counts
acmul = 0

for instance in validation_data.Data.iloc:
  # Check for the existence of the Value field and other fields; default to an empty list if missing
  value = instance['Answer'].get('Value', '') # Use an empty string if 'Value' is missing
  normalized_value = instance['Answer'].get('NormalizedValue', '') # Use an empty string if 'NormalizedValue' is missing
  aliases = instance['Answer'].get('Aliases', []) # Use an empty list if 'Aliases' is missing
  normalized_aliases = instance['Answer'].get('NormalizedAliases', []) # Use an empty list if 'NormalizedAliases' is missing

  # Combine all available fields into one list of strings
  all_text = [value, normalized_value] + aliases + normalized_aliases
  # Tokenize each text and calculate token counts
  token_counts = [len(tokenizer.tokenize(text)) for text in all_text]
  # Compute average token count
  average_tokens = np.mean(token_counts)
  # Add the average token count to the accumulator
  acmul += average_tokens

# Calculate the overall average token count across all instances in the dataset
final_average_tokens = acmul / len(validation_data)

print(f"The average number of tokens for an answer is: {final_average_tokens:.2f}")


In [24]:
# Define the sampling parameters
sampling_params = SamplingParams(
    n = 1,
    top_p=0.9,
    temperature=0,
    repetition_penalty=1.2,
    max_tokens=5,     # Maximum number of tokens
)

In [9]:
prompt_in_chat_format = [
    {
        "role": "system",
        "content": """Answer the question with only one word or the simplest possible response (e.g., a single number or a single word).
Do NOT generate sentences, explanations, or additional context.
Stop immediately after providing the answer. Do not generate any further words or tokens.
If the context does not provide any useful information, answer the question based on your own knowledge.
I am going to provide you five examples:

Question: What is the capital of Kenya?
Answer: Nairobi
---
Question: What was the name of the pig leader in George Orwell's Animal Farm?
Answer: Napoleon
---
Question: Which artist created the Katzenjammer Kids?
Answer: Rudolph Dirks
---
Question: Who was Geena Davis's husband when they made the loss-maker Cutthroat Island?
Answer: Renny Harlin
---
Question: Who was married to Spandau Ballet's Gary Kemp and later to Jude Law?
Answer: Sadie Frost

"""
    },
    {
        "role": "user",
        "content": """Context:
{context}
---
Now here is the question you need to answer.

Question: {question}"""
    },
]

RAG_PROMPT_TEMPLATE = tokenizer.apply_chat_template(
    prompt_in_chat_format, tokenize=False, add_generation_prompt=True
)
print(RAG_PROMPT_TEMPLATE)

System: Answer the question with only one word or the simplest possible response (e.g., a single number or a single word).
Do NOT generate sentences, explanations, or additional context.
Stop immediately after providing the answer. Do not generate any further words or tokens.
If the context does not provide any useful information, answer the question based on your own knowledge.
I am going to provide you five examples:

Question: What is the capital of Kenya?
Answer: Nairobi
---
Question: What was the name of the pig leader in George Orwell's Animal Farm?
Answer: Napoleon
---
Question: Which artist created the Katzenjammer Kids?
Answer: Rudolph Dirks
---
Question: Who was Geena Davis's husband when they made the loss-maker Cutthroat Island?
Answer: Renny Harlin
---
Question: Who was married to Spandau Ballet's Gary Kemp and later to Jude Law?
Answer: Sadie Frost


User: Context:
{context}
---
Now here is the question you need to answer.

Question: {question}
Assistant: 


In [ ]:
question = "Which Lloyd Webber musical premiered in the US on 10th December 1993?"
context = ""


prompt = [RAG_PROMPT_TEMPLATE.format(question = question, context = context)]
outputs = model.generate(prompt, sampling_params)

for output in outputs:
    generated_text = output.outputs[0].text
    prompt = output.prompt
    print(f"Question: {question!r}")
    print(f"Prompt: {prompt!r}")
    print(f"Generated text: {generated_text!r}")

In [ ]:
#Context provided by the embedding model

question = "Which Lloyd Webber musical premiered in the US on 10th December 1993?"

relevant_docs = KNOWLEDGE_VECTOR_DATABASE.similarity_search(question, k=3)
relevant_docs = [doc.page_content for doc in relevant_docs]
context = "\nExtracted documents:\n"
context += "".join(
            [f"Document {str(i)}:::\n" + doc for i, doc in enumerate(relevant_docs)]
        )

prompt = [RAG_PROMPT_TEMPLATE.format(question = question, context = context)]
outputs = model.generate(prompt, sampling_params)

for output in outputs:
    generated_text = output.outputs[0].text
    prompt = output.prompt
    print(f"Question: {question!r}")
    print(f"Prompt: {prompt!r}")
    print(f"Generated text: {generated_text!r}")

In [ ]:
# from ragatouille import RAGPretrainedModel

In [10]:
# Main function to answer questions using RAG
def answer_with_rag(
    questions: List[str],
    llm,
    prompt_template,
    sampling_params: dict,
    knowledge_index=None,
    embedding_model=None,
    reranker = None,
    # reranker: Optional[RAGPretrainedModel] = None,
    num_retrieved_docs: int = 30,
    num_docs_final: int = 3,
    printing: bool = True,
    retriever: str = "faiss"
) -> List[str]:
    """
    Main function for answering questions using a Retrieval-Augmented Generation (RAG) pipeline.
    """
    # Ensure that questions are in list format
    if isinstance(questions, str):
        questions = [questions]

    # Step 1: Generate embeddings for all questions in one go if using 'faiss'
    if retriever == "faiss":
        if embedding_model is None or knowledge_index is None:
            raise ValueError("For 'faiss' retriever, 'embedding_model' and 'knowledge_index' must be provided.")
        
        # Calculate all embeddings for the questions at once
        embeddings = embedding_model.embed_documents(questions)

    # Step 2: Retrieve contexts for each question
    contexts = []
    relevant_docs_list = []
    
    for idx, question in enumerate(questions):
        # Use the precomputed embedding for each question if using 'faiss'
        embedding = embeddings[idx] if retriever == "faiss" else None
        context, relevant_docs = retrieve(
            question,
            embedding=embedding,
            knowledge_index=knowledge_index if retriever == "faiss" else None,
            reranker=reranker,
            num_retrieved_docs=num_retrieved_docs,
            num_docs_final=num_docs_final,
            printing=printing,
            retriever=retriever,
        )
        contexts.append(context)
        relevant_docs_list.append(relevant_docs)

    # Step 3: Generate answers using the LLM model
    if printing:
        print("=> Generating answers...")
    answers = read(llm, sampling_params, prompt_template, contexts, questions)

    return answers, relevant_docs_list

In [11]:
def silent_rerank(reranker, question, relevant_docs, k):
    # Create a temporary buffer to capture stdout and stderr
    with io.StringIO() as buf, contextlib.redirect_stdout(buf), contextlib.redirect_stderr(buf):
        # Call the rerank function
        ranked_docs = reranker.rerank(question, relevant_docs, k=k)
    return ranked_docs

def retrieve(
    question: str,
    embedding: Optional[List[float]] = None,
    knowledge_index=None,
    reranker = None,
    # reranker: Optional[RAGPretrainedModel] = None,
    num_retrieved_docs: int = 30,
    num_docs_final: int = 3,
    printing: bool = True,
    retriever: str = "faiss"
) -> Tuple[str, List[str]]:
    """
    Retrieves and optionally reranks documents from the knowledge index.
    """
    if retriever not in {"faiss", "bm25", None}:
        raise ValueError(f"Unsupported retriever: {retriever}")

    # Step 1: Retrieve initial documents if a retriever is used
    if retriever == "faiss" or retriever == "bm25":
        if printing:
            print("=> Retrieving documents...")

        relevant_docs = []

        if retriever == "faiss":
            if embedding is None or knowledge_index is None:
                raise ValueError("For 'faiss' retriever, 'embedding' and 'knowledge_index' must be provided.")
            # Perform search using the precomputed embedding
            relevant_docs = knowledge_index.similarity_search_by_vector(embedding, k=num_retrieved_docs)
            relevant_docs = [doc.page_content for doc in relevant_docs]  # Extract only the content
        elif retriever == "bm25":
            # Retrieve documents using BM25
            retrieved_docs = bm25_retrieve(question, k=num_retrieved_docs)
            relevant_docs = [doc.page_content for _, _, doc in retrieved_docs]

        # Step 2: Optionally rerank the retrieved documents
        if reranker:
            if printing:
                print("=> Reranking documents...")
            relevant_docs = silent_rerank(reranker, question, relevant_docs, k=num_docs_final)
            relevant_docs = [doc["content"] for doc in relevant_docs]

        # Limit the number of final documents to the desired count
        relevant_docs = relevant_docs[:num_docs_final]

        # Step 3: Build the context for the LLM model
        context = "\nExtracted documents:\n" + "".join(
            [f"Document {i}:::\n{doc}\n" for i, doc in enumerate(relevant_docs)]
        )
    else:
        # If no retriever is provided, set the context as empty
        context = ""
        relevant_docs = [""]

    return context, relevant_docs

def read(llm, sampling_params, prompt_template, contexts, questions):
    """
    Generates answers from the LLM by formatting the question-context pairs into prompts.
    """
    # Format prompts by combining questions and contexts
    prompts = [prompt_template.format(question=q, context=c) for q, c in zip(questions, contexts)]
    
    # Generate answers using the LLM
    outputs = llm.generate(prompts, sampling_params)
    
    # Extract the generated text from the outputs
    outputs = [output.outputs[0].text for output in outputs]
    
    return outputs

In [ ]:
validation_data_batch = validation_data[0:10] 
questions = [instance["Question"] for instance in validation_data_batch["Data"]]

answers, _ = answer_with_rag(
    questions= questions,
    llm = model,
    prompt_template = RAG_PROMPT_TEMPLATE,
    sampling_params = sampling_params,
    knowledge_index= None,
    embedding_model= None,
    reranker = None,
    # reranker = RERANKER,
    num_retrieved_docs = 30,
    num_docs_final = 3,
    printing = True,
    retriever = "bm25"
)

In [ ]:
for q,a in zip(questions, answers):
    print("Question: ", q)
    print("Answer: ", a)

In [ ]:
question = "Where was born the Queen Elizabeth II?"

answers, relevant_docs_list = answer_with_rag(
    questions= question,
    llm = model,
    prompt_template = RAG_PROMPT_TEMPLATE,
    sampling_params = sampling_params,
    knowledge_index= KNOWLEDGE_VECTOR_DATABASE, 
    embedding_model= embedding_model,
    reranker = None, # If you don't want to use the reranker, set reranker = None
    num_retrieved_docs = 30,
    num_docs_final = 3,
    printing = True,
    retriever = "bm25" # Retriever options: "bm25", "faiss", None
)

print("Question: ", question)
print("Answer: ", answers[0])
print("Relevant Docs: ", relevant_docs_list[0])

In [ ]:
validation_data_batch = validation_data[0:10] 
questions = [instance["Question"] for instance in validation_data_batch["Data"]]

answers, _ = answer_with_rag(
    questions= questions,
    llm = model,
    prompt_template = RAG_PROMPT_TEMPLATE,
    sampling_params = sampling_params,
    knowledge_index= KNOWLEDGE_VECTOR_DATABASE, 
    embedding_model= embedding_model,
    reranker = None, # If you don't want to use the reranker, set reranker = None
    num_retrieved_docs = 30,
    num_docs_final = 3,
    printing = False,
    retriever = "faiss" # Retriever options: "bm25", "faiss", None
)

for q,a in zip(questions, answers):
    print("Question: ", q)
    print("Answer: ", a)

In [12]:
!git clone https://github.com/mandarjoshi90/triviaqa.git
!pip install -q -r /kaggle/working/triviaqa/requirements.txt

Cloning into 'triviaqa'...
remote: Enumerating objects: 70, done.
remote: Total 70 (delta 0), reused 0 (delta 0), pack-reused 70 (from 1)
Receiving objects: 100% (70/70), 20.60 KiB | 5.15 MiB/s, done.
Resolving deltas: 100% (28/28), done.


In [14]:
def perform_inference_p(instances, retriever, embedding_model=None, knowledge_index=None):
    # Processes a batch of instances to generate answers
    questions = [instance["Question"] for instance in instances]
    question_ids = [instance["QuestionId"] for instance in instances]

    # Generates answers using `answer_with_rag`
    '''
    answer_with_rag( 
    questions: List[str],
    llm,
    prompt_template,
    sampling_params: dict,
    knowledge_index=None,
    embedding_model=None,
    reranker = None,
    # reranker: Optional[RAGPretrainedModel] = None,
    num_retrieved_docs: int = 30,
    num_docs_final: int = 3,
    printing: bool = True,
    retriever: str = "faiss"
)
    '''
    responses, _ = answer_with_rag(
        questions= questions,
        llm = model,
        prompt_template = RAG_PROMPT_TEMPLATE,
        sampling_params = sampling_params,
        knowledge_index=knowledge_index,
        embedding_model=embedding_model,
        reranker=None,
        printing=False,
        retriever=retriever,
    )

    torch.cuda.empty_cache()

    # Associates the answers with their respective IDs
    results = [{"QuestionId": qid, "Answer": answer} for qid, answer in zip(question_ids, responses)]
    return results

def parallel_inference(validation_data, retriever="None", embedding_model=None, knowledge_index=None):
    # Initialize structures for predictions and TriviaQA data
    predictions = {}
    triviaqa_instances = {
        "Data": [],
        "Domain": "Wikipedia",
        "VerifiedEval": False,
        "Version": 1.0,
    }

    # Call perform_inference_p
    results = perform_inference_p(validation_data["Data"], retriever, embedding_model, knowledge_index)

    # Store the predictions
    for result in results:
        question_id = result["QuestionId"]
        answer = result["Answer"]
        predictions[question_id] = answer

    # Add the original instances to the TriviaQA set
    triviaqa_instances["Data"].extend(validation_data["Data"])

    return predictions, triviaqa_instances

In [15]:
# Example from the repository
!cd /kaggle/working/triviaqa && python3 -m evaluation.triviaqa_evaluation --dataset_file /kaggle/working/triviaqa/samples/triviaqa_sample.json --prediction_file /kaggle/working/triviaqa/samples/sample_predictions.json

Missed question tc_33--35/35_995.txt will receive score 0.
{'exact_match': 50.0, 'f1': 50.0, 'common': 1, 'denominator': 2, 'pred_len': 1, 'gold_len': 2}


In [ ]:
# 已经传进input了不用手动生成，不用跑这个
RAW_KNOWLEDGE_BASE_TEST = getDocs(test_data)

# Processing the RAW_KNOWLEDGE_BASE_TEST (test_data)
docs_processed_test = split_documents(
    512, # We choose a chunk size adapted to our model
    RAW_KNOWLEDGE_BASE_TEST,
    tokenizer_name=EMBEDDING_MODEL_NAME,
)

# Save the object
'''
with open("/kaggle/working/docs_processed_test.pkl", "wb") as file:
    pickle.dump(docs_processed_test, file)

# Create the vector database of document embeddings
KNOWLEDGE_VECTOR_DATABASE_TEST = FAISS.from_documents(
   docs_processed_test, embedding_model, distance_strategy=DistanceStrategy.COSINE
)
'''
#Save the vector database in a file
faiss_index_path_test = "/kaggle/working/knowledge_vector_database-test"
KNOWLEDGE_VECTOR_DATABASE_TEST.save_local(faiss_index_path_test)



In [18]:
# Load the pre-created vector database
# 改变数据库
# faiss_index_path = "/kaggle/input/faiss-500/Faiss"
# KNOWLEDGE_VECTOR_DATABASE_TEST = FAISS.load_local(faiss_index_path, embedding_model, allow_dangerous_deserialization=True)
faiss_index_path = "/kaggle/input/faiss-500/Faiss"
KNOWLEDGE_VECTOR_DATABASE_TEST = FAISS.load_local(faiss_index_path, embedding_model, allow_dangerous_deserialization=True)

# Load the test documents processed
with open("/kaggle/input/testset/pre-generated files/test_docs_processed.pkl", "rb") as file:
    docs_processed = pickle.load(file)

# Load the tokenized corpus
#with open('/kaggle/input/bm25-retriever/bm25_retriever.pkl', 'rb') as f:
#    loaded_tokenized_corpus = pickle.load(f)

# Reinitialize BM25 retriever using the loaded corpus
# bm25 = BM25Okapi(loaded_tokenized_corpus)

In [ ]:
#print(f"\nStarting retrieval for {user_query=}...")
#retrieved_docs = KNOWLEDGE_VECTOR_DATABASE_TEST.similarity_search(query=user_query, k=5)
#print(
#    "\n==================================Top 1 document=================================="
#)
#print(retrieved_docs[0].page_content)
#print("==================================Metadata==================================")
#print(retrieved_docs[0].metadata)
#print(
#    "\n==================================Top 2 document=================================="
#)
#print(retrieved_docs[1].page_content)
#print("==================================Metadata==================================")
#print(retrieved_docs[1].metadata)

In [17]:
# 定义文件夹路径
folder_path = '/kaggle/working/result_500'
 
# 创建文件夹
if not os.path.exists(folder_path):
    os.makedirs(folder_path)
    print(f"Folder '{folder_path}' created successfully.")
else:
    print(f"Folder '{folder_path}' already exists.")

Folder '/kaggle/working/result_500' created successfully.


In [25]:
batch_size = 100
for i in range(0, len(test_data), batch_size):

    data = test_data.iloc[i:i+batch_size]
    predictions, triviaqa_instances = parallel_inference(data, retriever="faiss", embedding_model=embedding_model,
        knowledge_index=KNOWLEDGE_VECTOR_DATABASE_TEST)
    file_path_instances = f'/kaggle/working/result_500/zephyr_faiss_triviaqa_instances_{i+batch_size}.json'
    file_path_predictions = f'/kaggle/working/result_500/zephyr_faiss_triviaqa_predictions_{i+batch_size}.json'
  
# Save the list of instances as s JSON file 
    with open(file_path_instances, 'w') as f:
        json.dump(triviaqa_instances, f, indent=4)
# Save the list of predictions as s JSON file
    with open(file_path_predictions , 'w') as f:
        json.dump(predictions, f, indent=4)

Chunks:   0%|          | 0/20 [00:00<?, ?it/s]

Chunks:   0%|          | 0/20 [00:00<?, ?it/s]

Chunks:   0%|          | 0/20 [00:00<?, ?it/s]

Chunks:   0%|          | 0/20 [00:00<?, ?it/s]

Chunks:   0%|          | 0/20 [00:00<?, ?it/s]

Chunks:   0%|          | 0/20 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [26]:
def combine(partialFilePath):
  num = 100
  predictions = {}
  triviaqa_instances = {
    "Data": [],
    "Domain": "Wikipedia",
    "VerifiedEval": False,
    "Version": 1.0,
  }

  for i in range(1,6):
    # Define the file path
    inst_path = partialFilePath + "_triviaqa_instances_" + str(i*num) + ".json"
    pred_path = partialFilePath + "_triviaqa_predictions_" + str(i*num) + ".json"
    # Load the instances from the JSON file
    with open(inst_path, 'r') as f:
      loaded_instances = json.load(f)
      triviaqa_instances['Data'] = triviaqa_instances['Data'] + loaded_instances['Data']
    # Load the predictions from the JSON file
    with open(pred_path, 'r') as f:
      loaded_predictions = json.load(f)
      predictions.update(loaded_predictions)
  return predictions, triviaqa_instances

In [27]:
predictions, triviaqa_instances = combine("/kaggle/working/result_500/zephyr_faiss")

In [28]:
# Save the list of instances as s JSON file
with open('/kaggle/working/result_500/zephyr_faiss_triviaqa_instances.json', 'w') as f:
    json.dump(triviaqa_instances, f, indent=4)
# Save the list of predictions as s JSON file 
with open('/kaggle/working/result_500/zephyr_faiss_triviaqa_predictions.json', 'w') as f:
    json.dump(predictions, f, indent=4) 

In [29]:
!cd /kaggle/working/triviaqa && python3 -m evaluation.triviaqa_evaluation --dataset_file /kaggle/working/result_500/zephyr_faiss_triviaqa_instances.json --prediction_file /kaggle/working/result_500/zephyr_faiss_triviaqa_predictions.json

em=0: Sunset ['sunset boulevard', 'sunset bulevard', 'west sunset boulevard', 'sunset blvd']
em=0: Bonar ['henry campbell bannerman', 'sir henry campbell bannerman', 'campbell bannerman']
em=0: None ['lauren becall', 'loren bacall', 'lauren becal', 'lauren bacall', 'betty j perske', 'betty perske', 'betty joan perske', 'bacall', 'betty joan perski']
em=0: Fiddler ['fiddler on roof', 'sprintze', 'anatevka', '2 life', 'fiddler on reoof']
em=0: Mutiny on the " ['mutiny on bounty history', 'hms bounty mutineers', 'bounty vessel', 'mutiny on bounty', 'thomas ledward']
em=0: Capote ['personhood theory', 'persons', 'person', 'perſon', 'perſons', 'person philosophical']
em=0: Actor ['master builder occupation', 'graziani corazza', 'architecht', 'clifford lawrie', 'architechts', 'architects', 'registered architect', 'hok canada inc', 'architect', 'stanford downey architects inc']
em=0: Stars on 45 Med ['stars on 45 song', 'stars on 45 single', 'stars on 45 medley', 'medley intro venus sugar sug

In [ ]:
def answer_without_rag(
    questions: List[str],
    llm,
    prompt_template,
    sampling_params: dict,
    printing: bool = True,
) -> List[str]:
    """
    Answer questions without any retrieval (i.e., vanilla LLM). 
    """
    if isinstance(questions, str):
        questions = [questions]

    # Prepare empty context (or prompt with no knowledge injection)
    contexts = [""] * len(questions)

    if printing:
        print("=> Generating answers without retrieval...")

    answers = read(llm, sampling_params, prompt_template, contexts, questions)
    return answers

In [ ]:
prompt_in_chat_format_no_context = [
    {
        "role": "system",
        "content": """Answer the question with only one word or the simplest possible response (e.g., a single number or a single word).
Do NOT generate sentences, explanations, or additional context.
Stop immediately after providing the answer. Do not generate any further words or tokens.
If the context does not provide any useful information, answer the question based on your own knowledge.
I am going to provide you five examples:

Question: What is the capital of Kenya?
Answer: Nairobi
---
Question: What was the name of the pig leader in George Orwell's Animal Farm?
Answer: Napoleon
---
Question: Which artist created the Katzenjammer Kids?
Answer: Rudolph Dirks
---
Question: Who was Geena Davis's husband when they made the loss-maker Cutthroat Island?
Answer: Renny Harlin
---
Question: Who was married to Spandau Ballet's Gary Kemp and later to Jude Law?
Answer: Sadie Frost

"""
    },
    {
        "role": "user",
        "content": """Now here is the question you need to answer.

Question: {question}"""
    },
]

NO_RAG_PROMPT_TEMPLATE = tokenizer.apply_chat_template(
    prompt_in_chat_format_no_context, tokenize=False, add_generation_prompt=True
)
print(NO_RAG_PROMPT_TEMPLATE)

In [ ]:
# 重定义该函数，适配不使用RAG的情况
def perform_inference_p(instances, retriever, embedding_model=None, knowledge_index=None):
    # Extract questions and their IDs
    questions = [instance["Question"] for instance in instances]
    question_ids = [instance["QuestionId"] for instance in instances]

    # 使用 RAG 或 非 RAG
    if retriever == "none":  # 小写 'none'，代表不使用RAG
        responses = answer_without_rag(
            questions=questions,
            llm=model,
            prompt_template=NO_RAG_PROMPT_TEMPLATE,
            sampling_params=sampling_params,
            printing=False,
        )
    else:
        responses, _ = answer_with_rag(
            questions=questions,
            llm=model,
            prompt_template=RAG_PROMPT_TEMPLATE,
            sampling_params=sampling_params,
            knowledge_index=knowledge_index,
            embedding_model=embedding_model,
            reranker=None,
            printing=False,
            retriever=retriever,
        )

    torch.cuda.empty_cache()

    # Return result dict
    results = [{"QuestionId": qid, "Answer": answer} for qid, answer in zip(question_ids, responses)]
    return results

In [ ]:
batch_size = 100
for i in range(0, len(test_data), batch_size):

    data = test_data.iloc[i:i+batch_size]
    predictions, triviaqa_instances = parallel_inference(data, retriever="none", embedding_model=embedding_model,
        knowledge_index=KNOWLEDGE_VECTOR_DATABASE_TEST)
    file_path_instances = f'/kaggle/working/result_no_rag/zephyr_faiss_triviaqa_instances_norag_{i+batch_size}.json'
    file_path_predictions = f'/kaggle/working/result_no_rag/zephyr_faiss_triviaqa_predictions_norag_{i+batch_size}.json'
  
# Save the list of instances as s JSON file
    with open(file_path_instances, 'w') as f:
        json.dump(triviaqa_instances, f, indent=4)
# Save the list of predictions as s JSON file
    with open(file_path_predictions , 'w') as f:
        json.dump(predictions, f, indent=4)

In [ ]:
def combine2(partialFilePath):
  num = 100
  predictions = {}
  triviaqa_instances = {
    "Data": [],
    "Domain": "Wikipedia",
    "VerifiedEval": False,
    "Version": 1.0,
  }

  for i in range(1,6):
    # Define the file path
    inst_path = partialFilePath + "_triviaqa_instances_norag_" + str(i*num) + ".json"
    pred_path = partialFilePath + "_triviaqa_predictions_norag_" + str(i*num) + ".json"
    # Load the instances from the JSON file
    with open(inst_path, 'r') as f:
      loaded_instances = json.load(f)
      triviaqa_instances['Data'] = triviaqa_instances['Data'] + loaded_instances['Data']
    # Load the predictions from the JSON file
    with open(pred_path, 'r') as f:
      loaded_predictions = json.load(f)
      predictions.update(loaded_predictions)
  return predictions, triviaqa_instances

In [ ]:
predictions, triviaqa_instances = combine2("/kaggle/working/result_no_rag/zephyr_faiss")

In [ ]:
# Save the list of instances as s JSON file
with open('/kaggle/working/result_no_rag/zephyr_faiss_triviaqa_instances.json', 'w') as f:
    json.dump(triviaqa_instances, f, indent=4)
# Save the list of predictions as s JSON file 
with open('/kaggle/working/result_no_rag/zephyr_faiss_triviaqa_predictions.json', 'w') as f:
    json.dump(predictions, f, indent=4) 

In [ ]:
!cd /kaggle/working/triviaqa && python3 -m evaluation.triviaqa_evaluation --dataset_file /kaggle/working/result_no_rag/zephyr_faiss_triviaqa_instances.json --prediction_file /kaggle/working/result_no_rag/zephyr_faiss_triviaqa_predictions.json

In [ ]:
# 40 minutes